## Bring together all elements of the PAM data analysis pipeline

This script is to test bringing together all elements of the PAM analysis pipeline. The required elements are:

- Folder containing the xpim files
- Folder containing the required subscripts:
     - pim2tiff.exe
     - Multi2singleframes.extract_frame (from src.data)
     - from scripts import copy_analyse_FvFm


If using this script **it is important to acknowledge the code contributors**:
- pim2tiff.exe was developed and provided by Oliver Meyerhoff of Walz (supplier of the PAM imager)
- The Multi2singleframes.extract_frame script is an edited version of that developed by Dominik Schneider et al. and published here (https://doi.org/10.1186/s13007-019-0546-1). See also GitHub for source scripts (https://github.com/CougPhenomics/ImagingPAMProcessing).
- The plantcv_current.yml environment file was also sourced from scripts written by Schneider et al.
- The FvFm calculations were carried out using scripts generated by PlantCV. See https://plantcv.readthedocs.io/en/stable/photosynthesis_analyze_fvfm/ for tutorial and GitHub (https://github.com/danforthcenter/plantcv/blob/main/plantcv/plantcv/photosynthesis/analyze_fvfm.py) for source scripts.

**Note:** This script can only be run using the "plantcv" environment, which contains the required Plantcv packages (as well as Python and subpackages). This can be changed later, by extracting the source codes for setting the threshold...

Before running locally, it is necessary to generate a conda environment, using the plantcv_current.yml file.

In [1]:
# Import all required scripts and functions:

import re
import os
import glob
import shutil
import pandas as pd
import numpy as np

# Required for setting threshold images (and subsequent cleaning)
from plantcv import plantcv as pcv
from skimage import filters

# Edited FvFm value extraction script
from scripts import Analyse_FvFm_new

# Convert tiff stack to individual files
from scripts import Multi2Singleframes
# Run executable
import subprocess
# Cut up tiff images
from PIL import Image

# Required for image export in ProcessImages.py script (not yet used here...)
import cv2 as cv2

In [2]:
# Get list of files to analyse
xpim_files = glob.glob("./input/xpim_files/*.xpim")
file_list = []
for file in xpim_files:
    bn = os.path.basename(file)
    bn = bn.strip(".xpim")
    file_list.append(bn)


# Convert xpim files to tif files. This utilises the "pim2tif.exe" executable.
subprocess.check_call(["./scripts/pim2tiff.exe", "./input/xpim_files"])

# Move tif files to a new folder
tif_files = glob.glob("./input/xpim_files/*.tif")
for file in tif_files:
    basename = os.path.basename(file)
    destination = "./input/tiff_files/" + basename
    shutil.move(file, destination)

In [3]:
# Separate the tif stacks in to individual images
tiff_frames = "./input/tiff_files/tiff_frames"

if not os.path.exists(tiff_frames):
    os.makedirs(tiff_frames) 
else: print("Warning: tif_frames directory already exists. Please check that no old tif files are in this directory!")
for tiff_stack in glob.glob("./input/tiff_files/*.tif"):
    Multi2Singleframes.extract_frames(tiff_stack, "./input/tiff_files/tiff_frames")

In [15]:
# Preset list for well positions.
# Note: Please check at the end of the data extraction process, that all the leaf area (and none from neighbouring wells) has been successfully captured.
Well_1 = (100,130,170,200)
Well_2 = (170,130,240,200)
Well_3 = (240,130,310,200)
Well_4 = (310,130,380,200)
Well_5 = (380,130,450,200)
Well_6 = (450,130,520,200)
Well_7 = (100,200,170,270)
Well_8 = (170,200,240,270)
Well_10 = (310,200,380,270)
Well_11 = (380,200,450,270)
Well_12 = (450,200,520,270)
Well_13 = (100,270,170,340)
Well_14 = (170,270,240,340)
Well_9 = (240,200,310,270)
Well_15 = (240,270,310,340)
Well_16 = (310,270,380,340)
Well_17 = (380,270,450,340)
Well_18 = (450,270,520,340)
Well_19 = (100,340,170,410)
Well_20 = (170,340,240,410)
Well_21 = (240,340,310,410)
Well_22 = (310,340,380,410)
Well_23 = (380,340,450,410)
Well_24 = (450,340,520,410)

# Create output pandas df
output_df = pd.DataFrame([], columns=["Plate", "Well", "FvFm"])
# For each file to analyse, generate and export the "fmax" image. This can be used for later debugging
for image_file in file_list:
    print(f"Analysing {image_file}")

    ####### Delete this block?? No need to save the images of the whole plate (when they already exist in input)? ###############
    # # Read in the TIF frames for the PAM image
    # fmax1, path, filename = pcv.readimage(f"./input/tiff_files/tiff_frames/{image_file}-2.tif", mode="native")
    
    # plate_image_output = "./output/plate_images"
    # if os.path.exists(plate_image_output):
    #     print("Error: output/plate_images already exists and cannot be overwritten. Please delete this directory and rerun script.")
    #     # exit()   
    # else:
    #     os.makedirs(plate_image_output)
    #     # Export the image files.
    #     # Note: these files can only be observed when open in ImageJ. When opening in Windows Photo Viewer, the image is black.
    #     cv2.imwrite(os.path.join(plate_image_output, image_file + '_fmax.tif'), fmax1)
    #############################################
    
    # # Make cropped image directories
    cropped_plate_output = "./debug/cropped_images"
    if not os.path.exists(cropped_plate_output):
        os.makedirs(cropped_plate_output) 
# Make output directories for the cropped images
# Individual directories are generated for ease of deletion later.
        os.makedirs(f"{cropped_plate_output}/fmin")
        os.makedirs(f"{cropped_plate_output}/fmax")
        os.makedirs(f"{cropped_plate_output}/fdark")
        os.makedirs(f"{cropped_plate_output}/threshold")
    else: print("Warning: cropped_images directory already exists. Please check that no old tif files are in this directory, as this could affect final FvFm results!") 

    # Create contrast image
    fmin_plate, path, filename = pcv.readimage(f"./input/tiff_files/tiff_frames/{image_file}-1.tif", mode="native")
    fmax_plate, path, filename = pcv.readimage(f"./input/tiff_files/tiff_frames/{image_file}-2.tif", mode="native")
    fdark_plate, path, filename = pcv.readimage(f"./input/tiff_files/tiff_frames/{image_file}-3.tif", mode="native")
    yen_threshold = filters.threshold_yen(image=fmax_plate)
    threshold_image = pcv.threshold.binary(gray_img=fmax_plate, threshold=yen_threshold, max_value=255, object_type='light')
    threshold_image = pcv.fill(threshold_image, size=5)
    cv2.imwrite(f"./output/{image_file}_threshold_image.tif", threshold_image)

    # Create images for each well
    for i in range(1,25):
        
        # For each plate image, the image is opened, cropped and the cropped image is saved
        with Image.open(f"./input/tiff_files/tiff_frames/{image_file}-1.tif") as plate_fmin:
            cropped_fmin = plate_fmin.crop(locals()["Well_"+str(i)])
            cropped_fmin.save(f"./debug/cropped_images/fmin/{image_file}_fmin_Well-{i}.tif", format=None)
            
        with Image.open(f"./input/tiff_files/tiff_frames/{image_file}-2.tif") as plate_fmax:
            cropped_fmax = plate_fmax.crop(locals()["Well_"+str(i)])
            cropped_fmax.save(f"./debug/cropped_images/fmax/{image_file}_fmax_Well-{i}.tif", format=None)
            
        with Image.open(f"./input/tiff_files/tiff_frames/{image_file}-3.tif") as plate_fdark:
            cropped_fdark = plate_fdark.crop(locals()["Well_"+str(i)])
            cropped_fdark.save(f"./debug/cropped_images/fdark/{image_file}_fdark_Well-{i}.tif", format=None)
        
        # Read back in using pcv functions (reads in images as numpy arrays)
        fmin, _, _ = pcv.readimage(f"./debug/cropped_images/fmin/{image_file}_fmin_Well-{i}.tif", mode="native")
        fmax, _, _ = pcv.readimage(f"./debug/cropped_images/fmax/{image_file}_fmax_Well-{i}.tif", mode="native")
        fdark, _, _ = pcv.readimage(f"./debug/cropped_images/fdark/{image_file}_fdark_Well-{i}.tif", mode="native")

        #Return threshold value based on Yen’s method.
        yen_threshold = filters.threshold_yen(image=fmax)
        # Use the threshold value to filter out highlighted plate areas
        threshold_image = pcv.threshold.binary(gray_img=fmax, threshold=yen_threshold, max_value=255, object_type='light')
        threshold_image = pcv.fill(threshold_image, size=5)
        # Carry out FvFm calculation
        part_Fv = Analyse_FvFm_new.analyze_fvfm(fdark=fdark, fmin=fmin, fmax=fmax, mask=threshold_image, bins=256, label="fluor")
        df = pd.DataFrame([[image_file, i, part_Fv]], columns = ["Plate", "Well", "FvFm"])
        output_df = pd.concat([output_df, df])
        # part_Fv, part_hist_fvfm = pcv.photosynthesis.analyze_fvfm(fdark=fdark, fmin=fmin, fmax=fmax, mask=threshold_image, bins=256, label="fluor")
        
        ################## Edit/remove this section? Introduction of bias!! #############################
        # To remove areas of wells still included after thresholding, pcv.fill removes objects that are less than specified size"
        # However, we need different levels of filtering for different wells. To rigorous filtering for some wells will remove plants from others!
        # pixel_count = cv2.countNonZero(threshold_image)
        # print(pixel_count)
        # if pixel_count > 500:
        #     threshold_image = pcv.fill(threshold_image, size=(pixel_count/5))
        #     threshold_image = pcv.erode(threshold_image,2,1)
        #     threshold_image = pcv.fill(threshold_image, pixel_count/10)
        # else: threshold_image = pcv.fill(threshold_image, size=1)
        ###################################################################################################

        # Export threshold images to debug folder
        cv2.imwrite(f"./debug/cropped_images/threshold/{image_file}_threshold_Well-{i}.tif", threshold_image)

output_df = output_df.reset_index().drop(columns="index")
output_df.to_csv("./output/FvFm_output.csv")

Analysing Plate1_3dpt
fvfm_median is 0.5845154845154845
fvfm_median is 0.715878042722305
fvfm_median is 0.7077337538603061
fvfm_median is 0.7321428571428571
fvfm_median is 0.7784810126582279
fvfm_median is 0.7009345794392523
fvfm_median is 0.6963562753036437
fvfm_median is 0.723404255319149
fvfm_median is 0.7588502506265664
fvfm_median is 0.6886792452830188
fvfm_median is 0.7505760368663594
fvfm_median is 0.6787878787878788
fvfm_median is 0.7247158262897115
fvfm_median is 0.7256637168141593
fvfm_median is 0.6837410638158326
fvfm_median is 0.6705882352941176
fvfm_median is 0.6923076923076923
fvfm_median is 0.7304964539007093
fvfm_median is 0.674074074074074
fvfm_median is 0.7125
fvfm_median is 0.746031746031746
fvfm_median is 0.7211155378486056
fvfm_median is 0.7453234265734265
fvfm_median is 0.7567567567567568


In [1]:
# Finally, remove unnecessary directories
os.rmdir(f"{cropped_plate_output}/fmin")
os.rmdir(f"{cropped_plate_output}/fdark")
os.rmdir(tiff_frames)
# fmax and threshold will be kept for debugging purposes.

NameError: name 'os' is not defined